# Exact Diagonalization of Hubbard Model

## Task

This notebook presents a sample code for diagonalizing the Hubbard model on a 6-site ladder:

<img src="./ladder.png" width="10%"/>

The Hamiltonian is as follows:
$$ H = -t\sum_{\{i, j\}, \sigma} (C^\dagger_{i,\sigma}C_{j,\sigma} + C^\dagger_{j,\sigma}C_{i,\sigma}) + U\sum_{i} n_{i\uparrow}n_{i\downarrow} + V\sum_{\{\{i, j\}\}}(n_{i\uparrow}+n_{i\downarrow})(n_{j\uparrow}+n_{j\downarrow})$$
where for this ladder we have
$$ \{i, j\} \in (0, 1), (2, 3), (4, 5), (0, 2), (2, 4), (1, 3), (3, 5)\\
\{\{i, j\}\} \in (0, 3), (2, 1), (2, 5), (4, 3) $$
and we take $t=1.0, U=1.3, V=-0.4.$

Our goal is to find the lowest 20 energy eigenvalues.


## Basic Bit Operations

First, let's define some basic bit operations for later use.


In [1]:
def countBinOnes(num):
    return bin(num).count('1')

def concatBins(left, right, leftShift=6):
    return (left << leftShift) + right

def splitBins(num, cutFromRight=6):
    return num >> cutFromRight, num % (1 << cutFromRight)

def readBit(num, n):
    return (num & (1 << n)) >> n

def flipBit(num, n):
    return num ^ (1 << n)

def pickBits(num, start, n):
    return (num & ((2 ** n - 1) << start)) >> start

## $U(1)$ & $Z_2$ Symmetry

We now use $U(1)$ symmetry to block-diagonalize $H$, dividing all possible $4^6=4096$ configurations into 
$$4096 = \sum_{0\le u\le 6, 0\le d \le 6} \binom{6}{u}\binom{6}{d}_{u\uparrow d\downarrow},$$
each having $u$ ups, $d$ downs and $6-u-d$ empty slots.

Further using $Z_2$ symmetry (exchanging ups and downs), we have:
$$4096 = 2\sum_{0\le d < u\le 6} \binom{6}{u}\binom{6}{d}_{u\uparrow d\downarrow} + \sum_{0\le u \le 6} \binom{6}{u}\binom{6}{u}_{u\uparrow u\downarrow} ,$$
leaving us at last $\frac{6\times 7}{2} + 6 = 21 + 6$ different small block matrices to diagonalize.

## Basis Representation

For $u$ ups, $d$ downs and $6-u-d$ empty slots, we adopt the convention that all spin-up fermions are put on the left side of spin-down fermions, and a state can be represented by a binary string of length 12. For example ($u=3, d=2$):
$$ C^\dagger_{0u}C^\dagger_{2u}C^\dagger_{3u}C^\dagger_{1d}C^\dagger_{3d}\ket{0} = \ket{001010_d~001101_u}.$$


## Lookup

To block diagonalize, we need a lookup function to locate a basis in its subspace. 

We build two tables, `tableToSub` and `tableFromSub`, to translate between subspace representation `[num of ups, num of downs, index]` and binary strings.


In [2]:
import numpy as np
def C(n,r):
    f = np.math.factorial
    return f(n) / f(r) / f(n-r)

In [3]:
def makeLookupTable():
    countUD = np.zeros((6+1, 6+1), dtype=int)

    tableToSub = [None for i in range(4**6)]
    tableFromSub = np.zeros((6+1, 6+1, int(C(6, 3))**2), dtype=int)

    for u in range(2**6):
        for d in range(2**6):
            numU = countBinOnes(u)
            numD = countBinOnes(d)

            du = concatBins(d, u, 6)

            subInfo = [numU, numD, countUD[numU, numD]]
            tableToSub[du] = subInfo
            tableFromSub[numU, numD, countUD[numU, numD]] = du

            countUD[numU, numD] += 1
            
    return tableToSub, tableFromSub, countUD

tableToSub, tableFromSub, subSpaceDim = makeLookupTable()

## Constructing and Diagonalizing Hamiltonian

We now turn to the counstruction of each block Hamiltonian matrix.

First, we construct $\{i, j\}$ and $\{\{i, j\}\}$.

In [4]:
ij = [(0, 1), (2, 3), (4, 5), (0, 2), (2, 4), (1, 3), (3, 5)] # i < j
iijj = [(0, 3), (2, 1), (2, 5), (4, 3)]
t = 1.0; U = 1.3; V = -0.4

Next, we construct and diagonalize the Hamiltonian for each $(u, d)$.
Note that the Hamiltonian consists of two parts:

1. Off-diagonal: $-t\sum_{\{i, j\}, \sigma} (C^\dagger_{i,\sigma}C_{j,\sigma} + C^\dagger_{j,\sigma}C_{i,\sigma})$
2. Diagonal: $U\sum_{i} n_{i\uparrow}n_{i\downarrow} + V\sum_{\{\{i, j\}\}}(n_{i\uparrow}+n_{i\downarrow})(n_{j\uparrow}+n_{j\downarrow})$

Furthermore, the anti-commutation relation of fermionic creation and annihilation operators must be taken with great care.

In [5]:
from scipy import sparse

eigenvalues = []
for u in range(6 + 1):
    for d in range(u + 1):

        HTo = []
        HFrom = []
        HValue = []

        for subFromBasis in range(subSpaceDim[u, d]):
            fromBasis = tableFromSub[u, d, subFromBasis]
            binD, binU = splitBins(fromBasis)
            
            # off-diagonal terms
            for i, j in ij:
                if readBit(binU, i) ^ readBit(binU, j):
                    HFrom.append(subFromBasis)
                    HTo.append(tableToSub[concatBins(binD, flipBit(flipBit(binU, i), j))][-1])
                    if readBit(binU, i):
                        HValue.append((-1) * (-t) * (-1)**(bin(pickBits(binU, 0, i)).count('1') + bin(pickBits(binU, 0, j)).count('1')))
                    else:
                        HValue.append(-t * (-1)**(bin(pickBits(binU, 0, i)).count('1') + bin(pickBits(binU, 0, j)).count('1')))

                if readBit(binD, i) ^ readBit(binD, j):
                    HFrom.append(subFromBasis)
                    HTo.append(tableToSub[concatBins(flipBit(flipBit(binD, i), j), binU)][-1])
                    if readBit(binD, i):
                        HValue.append((-1) * (-t) * (-1)**(bin(pickBits(binD, 0, i)).count('1') + bin(pickBits(binD, 0, j)).count('1')))
                    else:
                        HValue.append(-t * (-1)**(bin(pickBits(binD, 0, i)).count('1') + bin(pickBits(binD, 0, j)).count('1')))

            # diagonal terms
            HFrom.append(subFromBasis)
            HTo.append(subFromBasis)
            diag = U * bin(binD & binU).count('1')
            for i, j in iijj:
                diag += V * (readBit(binD, i) + readBit(binU, i)) * (readBit(binD, j) + readBit(binU, j))

            HValue.append(diag)
        
        # construct the Hamiltonian
        H = sparse.coo_matrix((HValue, (HTo, HFrom)), shape=(subSpaceDim[u, d], subSpaceDim[u, d])).tocsc()

        # diagonalize
        # making use of Z_2 symmetry
        if u == d:
            eigenvalues += np.real(np.linalg.eig(H.toarray())[0]).tolist()
        else:
            eigenvalues += np.real(np.linalg.eig(H.toarray())[0]).tolist() * 2
        

Now we can sort the list and find the lowest 20 energy eigenvalues:


In [6]:
print(sorted(eigenvalues)[:20])

[-7.545086897958434, -7.146815786445275, -7.146815786445275, -7.011488775864684, -7.011488775864684, -6.850656697153924, -6.850656697153883, -6.850656697153883, -6.739803377152178, -6.739803377152178, -6.660322380974283, -6.468360576966555, -6.41936847170763, -6.3938282455948166, -6.393828245594808, -6.393828245594808, -6.382465702357777, -6.382465702357777, -6.379674497979439, -6.379674497979439]


It can be readily seen here that the $Z_2$ symmetry can protect degeneracies from numerical instability.